In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [33]:
import pandas as pd
import numpy as np

from xgboost import plot_importance
import matplotlib.pyplot as pyplot

from preprocess import preprocess
from train import train_eval_model

In [182]:
skip = ['product_cd_dummies', 'card_type_dummies', 'device_type_dummies']
trs_new, test_set = preprocess(test=True, skip=skip)
trs_new.isna().sum()

train
date_sin


/home/dan/Projects/kaggle-ieee-cis-fraud-detection/env/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


amount


/home/dan/Projects/kaggle-ieee-cis-fraud-detection/env/lib/python3.6/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


product_cd_dummies
card_type_dummies
device_type_dummies
date_sin
amount
product_cd_dummies
card_type_dummies
device_type_dummies


TransactionID    0
isFraud          0
phase_sin        0
phase_cos        0
amt_scale        0
amt_diff         0
amt_round_1      0
amt_round_10     0
amt_round_100    0
dtype: int64

In [183]:
trs_new.isna().sum()

TransactionID    0
isFraud          0
phase_sin        0
phase_cos        0
amt_scale        0
amt_diff         0
amt_round_1      0
amt_round_10     0
amt_round_100    0
dtype: int64

In [ ]:
model, auc = train_eval_model(trs_new)
print(auc)

Fitting 4 folds for each of 1 candidates, totalling 4 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
plot_importance(model)

In [ ]:
test_set = test_set[trs_new.drop(columns=['isFraud']).columns]
test = test_set.drop(columns=['TransactionID'])

In [ ]:
preds = model.predict_proba(test)[:, 1]
preds

In [138]:
labels = pd.read_csv('data/train_transaction.csv')
labels = labels.loc[labels.TransactionDT >= 13000000, 'isFraud']

In [ ]:
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(labels, preds)
auc

In [86]:
submission = test_set[['TransactionID']]
submission.loc[:, 'isFraud'] = preds
print(submission)
submission.to_csv('submission2.csv', index=False)

        TransactionID       isFraud
0             3663549  3.701011e-07
1             3663550  1.354091e-01
2             3663551  1.094881e-05
3             3663552  1.035023e-03
4             3663553  1.307248e-03
...               ...           ...
506686        4170235  2.402308e-03
506687        4170236  2.808629e-03
506688        4170237  1.534056e-02
506689        4170238  6.811960e-08
506690        4170239  3.963507e-02

[506691 rows x 2 columns]
